# Calculate flow using actual gaussians vs known values

In the LAP implementation, there is a part where the calculated coeffients are multiplied by various sums over the kernels. In the LAP paper this is the "extract displacement" part. These sums, however have a known value, either 1 or 0.

Here I compare the quality of the LAP alg with performing the sums over the generated kernels and without (inputing the actual known value).

In [2]:
using LAP_julia, TimerOutputs, DataFrames, TableView, JLD2, FileIO, PyPlot, ImageFiltering
import LAP_julia: average_dicts, print_dict, compare_dicts, lap, single_lap, prepare_gaussian_filters, window_sum3!, multi_mat_div2, inpaint_nans!, smooth_with_gaussian!

┌ Info: Precompiling LAP_julia [797d5b5c-2e9a-429f-9c9f-727808915e33]
└ @ Base loading.jl:1273


LAP_julia succesfully loaded!


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-18053683368423682958\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-17440337677823750834\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-18053683368423682958\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16647633871734557823\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-14883657046187199939\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16647633871734557823\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## Gather data

In [3]:
# prepare new dataframe
calculate_flow_df = DataFrame(
    index = Int[],
    mode = Symbol[],
    img_size = Int[],
    whs = Int[],
    timer = TimerOutput[],
    results = Dict{String,Float64}[]
    )

window_half_sizes = cat(collect(1:51), collect(61:10:101), collect(141:40:381), dims=1);
img_sizes = [50, 100, 200, 400, 800, 1600];
modes = [:known_value; :calculation];

In [4]:


# fill dataframe
df = calculate_flow_df
let index = 0
    for img_size in img_sizes
        img, imgw, flow = gen_init(:chess, chess_args=[25, img_size/25])

        # limit the max window half size to 1/4 the image
        whs_limit = img_size/4
        whs_modified = filter(x -> x <= whs_limit, window_half_sizes)

        for whs in whs_modified
            for mode in modes

                window = [whs * 2 + 1, whs * 2 + 1]
                timer = TimerOutput("reg alg: lap")
                # run once:
                flow_est, source_reg, timer, results = test_registration_alg(lap, img, imgw, flow, [whs, window],
                    Dict(:timer => timer, :new_feature => mode == :calculation ? false : true), timer=timer, display=false)
                
                index = index + 1
                println("at index: ", index, " img_size:", img_size, " whs: ", whs)
                push!(df, Dict(:index => index,
                               :mode => mode,
                               :img_size => img_size,
                               :whs => whs,
                               :timer => timer,
                               :results => results))
            end
        end
    end
end

@save "calc_vs_known_df.jld2" df 

InterruptException: InterruptException:

In [22]:
new_avg_dicts = []
old_avg_dicts = []

for img_size in img_sizes

    n = count(k -> k < (img_size/4), window_half_sizes)

    new_dicts = df[(df.mode .== :known_value) .& (df.img_size .== img_size), :][:, :results][1:n]
    old_dicts = df[(df.mode .== :calculation) .& (df.img_size .== img_size), :][:, :results][1:n]

    new_avg = average_dicts(new_dicts)
    old_avg = average_dicts(old_dicts)

    new_avg_dicts = cat(new_avg_dicts, new_avg, dims=1)
    old_avg_dicts = cat(old_avg_dicts, old_avg, dims=1)

    print_dict(new_avg, "new_avg, with img_size: " * string(img_size))
    print_dict(old_avg, "old_avg, with img_size: " * string(img_size))
end

println("************************************************")
println("*****  calculation mode / known value mode *****")
println("************************************************")

for (old_dict, new_dict) in zip(old_avg_dicts, new_avg_dicts)
    comparison = compare_dicts(old_dict, new_dict)
    print_dict(comparison)
end
    

alksdjf
alksdjf
-------------------------
  new_avg, with img_size: 50
-------------------------
  mse            | 0.228
  rmse           | 0.475
  time           | 0.029
  ncc            | 0.535
  angle-rmse     | 55.828
  angle-mae      | 44.603
  mae            | 0.239
-------------------------
  old_avg, with img_size: 50
-------------------------
  mse            | 0.246
  rmse           | 0.495
  time           | 0.104
  ncc            | 0.498
  angle-rmse     | 45.494
  angle-mae      | 33.766
  mae            | 0.257
alksdjf
alksdjf
-------------------------
  new_avg, with img_size: 100
-------------------------
  mse            | NaN
  rmse           | NaN
  time           | 0.044
  ncc            | NaN
  angle-rmse     | NaN
  angle-mae      | NaN
  mae            | NaN
-------------------------
  old_avg, with img_size: 100
-------------------------
  mse            | NaN
  rmse           | NaN
  time           | 0.047
  ncc            | NaN
  angle-rmse     | NaN
  angle-

## Conclusion

It is always better to use the know values.

## Functions to be tested
Changed in LAP_julia, saved here.

In [5]:
function lap(img::Image,
             imgw::Image,
             fhs,
             window_size;
             timer::TimerOutput=TimerOutput("lap"),
             display::Bool=false,
             new_feature=true)

    @timeit_debug timer "lap" begin
        classic_estim = single_lap(img, imgw, fhs, window_size, timer=timer, display=display, new_feature=new_feature)
    end
    @timeit_debug timer "inpainting" begin
        inpaint_nans!(classic_estim)
    end
    @timeit_debug timer "smoothing" begin
        smooth_with_gaussian!(classic_estim, window_size)
    end
    @timeit_debug timer "generate source_reg" begin
        source_reg = warp_img(imgw, -real(classic_estim), imag(classic_estim))
    end
    # if display
    #     print_timer(timer)
    # end
    return classic_estim, source_reg
end


function single_lap(image_1::Image,
                    image_2::Image,
                    filter_half_size::Integer,
                    window,
                    filter_num::Integer=3;
                    timer::TimerOutput=TimerOutput("LAP"),
                    display::Bool=false,
                    new_feature=true)
    
    pixel_count = length(image_1)
    image_size = size(image_1)
    filter_size = 2 * filter_half_size + 1

    # prepare 2D filter basis for later use:
    basis = similar(image_1, (filter_size, filter_size, filter_num))

    # dif = (image forward - image backward) of each filter
    dif = similar(image_1, (image_size..., filter_num))

    @timeit_debug timer "filtering" begin
        if filter_num == 3
            # get the relevant forward and backward filters
            forward_ker, backward_ker = prepare_gaussian_filters(filter_half_size)

            # temporary place to store filtered images
            tmp_filtered_1 = similar(image_1)
            tmp_filtered_2 = similar(image_2)

            # basis 1 - (gaus, gaus) both forward and backward
            basis[:, :, 1] = broadcast(*, forward_ker[1]...)
            imfilter!(tmp_filtered_1, image_1, forward_ker[1], "symmetric")
            imfilter!(tmp_filtered_2, image_2, backward_ker[1], "symmetric")
            dif[:, :, 1] = tmp_filtered_2 - tmp_filtered_1

            # basis 2 - (gaus, gaus_der_flip) as forward, (gaus, gaus_der) as backward
            basis[:, :, 2] = broadcast(*, forward_ker[2]...)
            imfilter!(tmp_filtered_1, image_1, forward_ker[2], "symmetric")
            imfilter!(tmp_filtered_2, image_2, backward_ker[2], "symmetric")
            dif[:, :, 2] = tmp_filtered_2 - tmp_filtered_1

            # basis 3 - (gaus_der_flip, gaus) as forward, (gaus_der, gaus) as backward
            basis[:, :, 3] = broadcast(*, forward_ker[3]...)
            imfilter!(tmp_filtered_1, image_1, forward_ker[3], "symmetric")
            imfilter!(tmp_filtered_2, image_2, backward_ker[3], "symmetric")
            dif[:, :, 3] = tmp_filtered_2 - tmp_filtered_1
        end
    end

    dif = reshape(dif, (:, filter_num))

    # prepare matrices for linear system of equations
    A = similar(image_1, (filter_num-1, filter_num-1, pixel_count))
    b = similar(image_1, (filter_num-1, pixel_count))

    @timeit_debug timer "prepare A and b" begin
        w = Int64.((window[1]-1)/2)
        for k in 1:filter_num-1
            for l in k:filter_num-1
                @timeit_debug timer "window sum part 1" @views window_sum3!(A[k, l, :], dif[:, k+1] .* dif[:, l+1], image_size, window)
                A[l, k, :] = A[k, l, :]
            end
            @timeit_debug timer "window sum part 2" @views window_sum3!(b[k, :], dif[:, k+1] .* dif[:, 1] .* (-1), image_size, window)
        end
    end

    # Perform Gauss elimination on all pixels in parallel:
    # coeffs will be of shape: pixel_count, filter_num-1
    @timeit_debug timer "multli mat div 2" begin
        @views coeffs = multi_mat_div2(A, b)
    end
    # adding ones so that all base filters have their coefficients even the first one
    all_coeffs = [ones(pixel_count) coeffs]

    # make a border mask. Is 0 if its in a border of filter_half_size size.
    window_half_size = Int64.((window .- 1) ./ 2)
    border_mask = parent(padarray(ones((image_size .- (2 .* window_half_size))...),
                    Fill(NaN, window_half_size, window_half_size)))
    @views all_coeffs = all_coeffs .* reshape(border_mask, (:, 1))

    k = (-filter_half_size:filter_half_size)

    # Get the displacement vector field from the filters
    u_est = similar(image_1, ComplexF64, image_size...)
    @timeit_debug timer "calculate flow" begin
        for n in 1:filter_num
            if new_feature
                @views u_est[:] = 2 .* ((im .* (.-1 .* all_coeffs[:, 3]) ./ all_coeffs[:, 1]) .+ ((.-1 .* all_coeffs[:, 2]) ./ all_coeffs[:, 1]));
            else
                u1_top = zeros(image_size);
                u1_bot = zeros(image_size);
                u2_top = zeros(image_size);
                u2_bot = zeros(image_size);
                
                @views u1_top[:] = u1_top[:] .- sum(transpose(basis[:, :, n]) * k) .* all_coeffs[:, n];
                @views u1_bot[:] = u1_bot[:] .+ sum(basis[:, :, n]) .* all_coeffs[:, n];

                @views u2_top[:] = u2_top[:] .- sum(basis[:, :, n] * k) .* all_coeffs[:, n];
                @views u2_bot[:] = u2_bot[:] .+ sum(basis[:, :, n]) .* all_coeffs[:, n];
                u_est = 2 .* ((im .* u1_top ./ u1_bot) .+ (u2_top ./ u2_bot));
            end
        end
    end

    # dont use estimations whose displacement is larger than the filter_half_size
    displacement = real(u_est).^2 .+ imag(u_est).^2
    displacement_mask = displacement .> filter_half_size^2
    u_est[displacement_mask] .= NaN .+ NaN .* 1im;

    if (display && getfield(LAP_julia, :timeit_debug_enabled)())
        print_timer(timer)
        println()
    end
    return u_est
end

function asses_source_reg_quality(target, source_reg; title="", display::Bool=true)
    functions = [ncc, mae, rmse, mse]
    names_short = ["ncc", "mae", "rmse", "mse"]
    vals = map(x -> x(target, source_reg), functions)
    names_vals_dict = Dict(names_short[i] => vals[i] for i in 1:4)
    return names_vals_dict
end

function asses_flow_quality(flow, flow_est; title="", display::Bool=true)
    functions = [angle_mae, angle_rmse, mae, mse]
    names_short = ["angle-mae", "angle-rmse", "flow_mae", "flow_rmse"]
    # names = ["angle mean absolute error", "angle root mean squared error", "mean absolute error", "root mean squared error"]
    vals = map(x -> x(flow, flow_est), functions)
    names_vals_dict = Dict(names_short[i] => vals[i] for i in 1:4)
    return names_vals_dict
end

# TODO fix

function average_dicts(dicts)
    avg_dict = Dict()
    bad_count = 0
    for dict in dicts
        for key in keys(dict)
            if dict[key] == NaN
                bad_count += 1
                break;
            end
            if !haskey(avg_dict, key)
                avg_dict[key] = dict[key]
            else
                avg_dict[key] += dict[key]
            end
        end
    end
    for key in keys(avg_dict)
        avg_dict[key] = avg_dict[key]/(length(dicts)-bad_count)
    end
    return avg_dict
end


function compare_dicts(old_dict, new_dict)
    comparation_dict = Dict()
    for key in keys(old_dict)
        if haskey(new_dict, key)
            comparation_dict[key] = old_dict[key]/new_dict[key]
        end
    end
    return comparation_dict
end


single_lap (generic function with 2 methods)

In [32]:
methods(lap)

# 1 method for generic function "lap":
[1] lap(img::Array{T,2} where T<:Real, imgw::Array{T,2} where T<:Real, fhs, window_size; timer, display, new_feature) in Main at /Users/MrTrololord/Google_Drive/cvut/bakalarka/LAP_julia/src/helpers.jl:167

In [ ]:
img, imgw, flow = gen_init()
lap(img, imgw, 12, [25,25], new_feature=false, display=false);